# Output Parser
- Output Parser는 LLM의 출력을 처리하여 애플리케이션에서 사용할 수 있는 적절한 형식으로 변환하는 역할을 한다. 
  - LLM이 생성한 결과(Raw text)를 분석하여 **특정 정보를 추출**하거나, **원하는 형식으로 재구성**하는 데 사용된다.
- Output parser를 통해 LLM이 응답하는 **비구조적 데이터를 구조화된 데이터로 변환**하여 후속 작업에 적합하게 만드는 데 사용된다.

## 주요 Output Parser
1. **CommaSeparatedListOutputParser**
    - 쉼표로 구분된 텍스트를 리스트로 변환
2. **JsonOutputParser**
    - JSON 형태로 받은 결과를 JSON 형식으로 변환
3. **PydanticOutputParser**
    - JSON 형태로 받은 결과를 Pydantic 모델 객체로 변환
4. **YamlOutputParser**
    - YAML 형태로 받은 응답을 pydantic 모델객체로 변환.
5. **StrOutputParser**
    - 모델의 출력결과를 문자열로 변환
- PydanticOutputParser, JsonOutputParser, YamlOutputParser는 Pydantic을 이용해 schema를 정의하고 이를 이용해 출력을 변환한다.
  
## 메소드
- parse(text:str)
  - LLM이 생성한 응답(text)을 받아 출력 구조에 맞게 변환하여 반환.
- get_format_instructions(): str
  - output parser가 입력받을 형식으로 LLM이 출력(응답) 하도록 하는 프롬프트를 반환한다.
  - LLM에 전송하는 프롬프트에 포함되어 출력 형식을 안내한다.
- [**Runnable**](05_chaing_LECL.ipynb#Runnable)을 상속받아 구현되어 invoke()를 이용해서 parsing 할 수있다.


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## StrOutputParser
- 모델(LLM)의 출력 결과를 string으로 변환하여 반환하는 output parser.
- Chat Model은  Message 객체에서 content 속성값을 추출하여 문자열로 반환한다.

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# output parser import
from langchain_core.output_parsers import StrOutputParser

prompt_template = ChatPromptTemplate.from_template(
    "한국의 {topic} 관련된 속담을 {count}개 알려줘. 목력형식으로 작성해줘."
)
model = ChatOpenAI(model="gpt-4o-mini")
# output parser
parser = StrOutputParser()

In [8]:
# 프롬프트 생성
prompt = prompt_template.invoke({"topic":"호랑이", "count":5})
# print(prompt.to_messages)

## LLM 모델에 요청
res = model.invoke(prompt)
# 응답 결과를 확인.
print(res)

## parser 를 이용해서 content 만 추출
result = parser.invoke(res)
print(result)

content='1. 호랑이 굴에 가야 호랑이 새끼를 잡는다.  \n2. 호랑이 담배 피우던 시절.  \n3. 호랑이 없는 곳에서 여우가 왕 노릇한다.  \n4. 호랑이와 함께 살면 호랑이처럼 된다.  \n5. 호랑이의 눈을 피하라.  ' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 32, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3de1288069', 'finish_reason': 'stop', 'logprobs': None} id='run-e646ff5a-805e-424c-8516-cd5800c12360-0' usage_metadata={'input_tokens': 32, 'output_tokens': 81, 'total_tokens': 113, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
1. 호랑이 굴에 가야 호랑이 새끼를 잡는다.  
2. 호랑이 담배 피우던 시절.  
3. 호랑이 없는 곳에서 여우가 왕 노릇한다.  
4. 호랑이와 함께 살면 호랑이처럼 된다.  
5. 호랑이의 눈을 피하라.  



## CommaSeparatedListOutputParser

- `,`를 구분자로 하는 항목들을 받아서 List로 반환한다.
  - "a,b,c" => ['a','b','c']

In [9]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
parser = CommaSeparatedListOutputParser() # 객체(만) 생성
txt = "이순신, 유관순, 강감찬, 안중근"
res = parser.invoke(txt)
print(type(res))
print(res)

<class 'list'>
['이순신', '유관순', '강감찬', '안중근']


In [11]:
# LLM 에 요구하는 응답형태의 형식을 prompt에 작성해서 전송해야함.
## 이 output parser 가 변환할 수 있는 형식의 문자열을 받기 위한 가이드이며, 이것을 **프롬프트**에 추가한다. 
print(parser.get_format_instructions())  # prompt에 넣어야할 guide를 반환해주는 코드.code

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [15]:
from langchain.prompts import PromptTemplate

parser = CommaSeparatedListOutputParser()
prompt_template = PromptTemplate(
    template="{subject}의 이름 다섯개를 나열해 주세요.\n{format_instruction}",    
    partial_variables={"format_instruction":parser.get_format_instructions()}
)
# artial_variables=dict : 템플릿 변수에 값을 prompt template을 만들면서 넣을 때 사용. (invoke)()에 넣지 않고.)
# print(prompt_template.template)

model = ChatOpenAI(model="gpt-4o-mini")

# prompt 생성 - {format_instruction} 변수에는 partial_variables에 설정한 값을 넣는다.
prompt = prompt_template.invoke({"subject":"과일"})
# print(prompt)
# LLM 에 요청
res = model.invoke(prompt)


In [16]:
# res.content.split(', ')
print(res.content)
result = parser.invoke(res)
print(result)

사과, 바나나, 오렌지, 포도, 키위
['사과', '바나나', '오렌지', '포도', '키위']


In [17]:
# 입력 
#   ->prompt template 
#       -> prompt 생성
#           -> LLM 응답
#               -> parser에 응답을 넣어줌
#                   -> 최종 응답을 만들어줌

## chain 생성 (pipeline)
chain = prompt_template | model | parser

result = chain.invoke({"subject":"자동차"})
print(result)

['현대 아반떼', '기아 K5', '토요타 캠리', '폭스바겐 골프', 'BMW 3시리즈']


In [18]:
'''{
    "이름":"",
    "나이":,
    "취미":["","",""]


}'''

'{\n    "이름":"",\n    "나이":,\n    "취미":[""]\n\n\n}'


## JsonOutputParser

- JSON 형태로 받은 응답을 dictionary로 반환
- JSON 형식을 정하려는 경우 [Pydantic](Pydantic.ipynb)을 이용해 JSON 스키마를 정의하여 JsonOutputParser 생성시 전달한다.
- LLM이 JSON Schema를 따르는 형태로 응답을 하게 하고 그 것을 JsonOutputParser는 Dictionary로 변환한다.



In [20]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
txt = '''{
    "이름":"홍길동",
    "나이":20,
    "취미":["게임","독서"]
}'''
res = parser.invoke(txt)
print(type(res))
print(res['이름'], res['나이'])
print(parser.get_format_instructions())

<class 'dict'>
홍길동 20
Return a JSON object.


In [39]:
parser = JsonOutputParser()
prompt_template = PromptTemplate(
    template="{name}에 대해서 설명해줘.\n{format_instruction}",
    partial_variables={"format_instruction":parser.get_format_instructions()}
)
model = ChatOpenAI(model="gpt-4o-mini")

prompt = prompt_template.invoke({"name":"아이폰"})
# print(prompt)
res = model.invoke(prompt)

In [40]:
print(res.content)
print("="*100)

result = parser.invoke(res)
print(type(result))
result

```json
{
  "device": {
    "name": "아이폰",
    "manufacturer": "애플",
    "first_release": "2007년",
    "latest_model": "아이폰 15",
    "operating_system": "iOS",
    "features": [
      "터치 스크린",
      "고급 카메라 시스템",
      "Face ID 및 Touch ID 보안 기능",
      "App Store를 통한 다양한 애플리케이션 다운로드",
      "iCloud를 통한 데이터 저장 및 동기화",
      "Apple Pay를 통한 모바일 결제 기능"
    ],
    "design": {
      "materials": ["유리", "알루미늄", "스테인리스 스틸"],
      "colors": ["블랙", "화이트", "레드", "골드", "퍼플"]
    },
    "accessories": [
      "AirPods",
      "Apple Watch",
      "MagSafe 충전기",
      "케이스 및 스크린 보호 필름"
    ],
    "target_audience": "일반 소비자, 비즈니스 사용자, 학생"
  }
}
```
<class 'dict'>


{'device': {'name': '아이폰',
  'manufacturer': '애플',
  'first_release': '2007년',
  'latest_model': '아이폰 15',
  'operating_system': 'iOS',
  'features': ['터치 스크린',
   '고급 카메라 시스템',
   'Face ID 및 Touch ID 보안 기능',
   'App Store를 통한 다양한 애플리케이션 다운로드',
   'iCloud를 통한 데이터 저장 및 동기화',
   'Apple Pay를 통한 모바일 결제 기능'],
  'design': {'materials': ['유리', '알루미늄', '스테인리스 스틸'],
   'colors': ['블랙', '화이트', '레드', '골드', '퍼플']},
  'accessories': ['AirPods', 'Apple Watch', 'MagSafe 충전기', '케이스 및 스크린 보호 필름'],
  'target_audience': '일반 소비자, 비즈니스 사용자, 학생'}}

In [41]:
result['device']

{'name': '아이폰',
 'manufacturer': '애플',
 'first_release': '2007년',
 'latest_model': '아이폰 15',
 'operating_system': 'iOS',
 'features': ['터치 스크린',
  '고급 카메라 시스템',
  'Face ID 및 Touch ID 보안 기능',
  'App Store를 통한 다양한 애플리케이션 다운로드',
  'iCloud를 통한 데이터 저장 및 동기화',
  'Apple Pay를 통한 모바일 결제 기능'],
 'design': {'materials': ['유리', '알루미늄', '스테인리스 스틸'],
  'colors': ['블랙', '화이트', '레드', '골드', '퍼플']},
 'accessories': ['AirPods', 'Apple Watch', 'MagSafe 충전기', '케이스 및 스크린 보호 필름'],
 'target_audience': '일반 소비자, 비즈니스 사용자, 학생'}

In [ ]:
# JSON 문자열의 구조 (schema- 스키마)를 정의
## 알고 싶은 정보가 무엇인지(json에 어떤 내용을 넣어야 하는지)를 설정.
## pydantic 라이브러리를 이용해서 정의

In [ ]:
# ctrl + ` : 터미널 실행
## pip show pydantic 설치만 확인함

In [51]:
# 스키마 정의
from pydantic import BaseModel, Field
class Item(BaseModel):
    # 스키마 를 class 변수로 정의 - 
    # 변수명 = key 값. 
    # type hint로 dict 로 변환될때 value의 타입
    name:str = Field(description="제품의 이름")
    info:str = Field(description="제품에 대한 소개 정보")
    price:int = Field(description="제품의 한국에서 가격")

parser = JsonOutputParser(pydantic_object=Item)
print(parser.get_format_instructions())

parser = JsonOutputParser()
prompt_template = PromptTemplate(
    template="{name}에 대해서 설명해줘.\n{format_instruction}",
    partial_variables={"format_instruction":parser.get_format_instructions()}
)
model = ChatOpenAI(model="gpt-4o-mini")

prompt = prompt_template.invoke({"name":"MAC mini M4"})
# print(prompt)
res = model.invoke(prompt)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "제품의 이름", "title": "Name", "type": "string"}, "info": {"description": "제품에 대한 소개 정보", "title": "Info", "type": "string"}, "price": {"description": "제품의 한국에서 가격", "title": "Price", "type": "integer"}}, "required": ["name", "info", "price"]}
```


In [52]:
print(res.content)
result = parser.invoke(res)

print(result)

```json
{
  "model": "Mac Mini M4",
  "release_date": "Expected Late 2023",
  "processor": {
    "type": "Apple M4",
    "architecture": "ARM64",
    "cores": 8,
    "performance_cores": 4,
    "efficiency_cores": 4,
    "GPU": "Integrated 10-core GPU"
  },
  "memory": {
    "options": [
      {
        "type": "LPDDR5",
        "sizes": ["8GB", "16GB", "32GB"]
      }
    ]
  },
  "storage": {
    "options": [
      {
        "type": "SSD",
        "sizes": ["256GB", "512GB", "1TB", "2TB"]
      }
    ]
  },
  "connectivity": {
    "ports": [
      "2 x Thunderbolt 4",
      "2 x USB-A",
      "HDMI 2.0",
      "Ethernet",
      "3.5mm audio jack"
    ],
    "wireless": [
      "Wi-Fi 6E",
      "Bluetooth 5.3"
    ]
  },
  "operating_system": "macOS Ventura",
  "dimensions": {
    "width": "7.7 inches",
    "depth": "7.7 inches",
    "height": "1.4 inches",
    "weight": "3.0 pounds"
  },
  "features": [
    "Compact design",
    "Energy-efficient performance",
    "Support for exter

## PydanticOutputParser

- JSON 형태로 받은 응답을 Pydantic 모델로 변환하여 반환한다.
- 구현은 JsonOutputParser와 동일한데 parsing 결과를 pydantic 모델로 반환한다.

In [59]:
from langchain_core.output_parsers import PydanticOutputParser
class Person(BaseModel):
    name: str = Field(description="사람의 이름")
    yob: int = Field(description="name의 사람이 태어난 년도")
    yob: int = Field(description="name의 사람이 사망한 년도")
    profile: str = Field(description="name의 사람에 대한 정보(profile)")

parser = PydanticOutputParser(pydantic_object=Person)
print(parser.get_format_instructions())         
# JsonoutputParser와 동일한 guide. JSON 으로 응답 -> Pydantic Model 객체로 변환.

prompt_template = PromptTemplate(
    template="{name}에 대해서 설명해주세요. \n{format_instruction}",
    partial_variables={"format_instruction": parser.get_format_instructions()}
)

prompt = prompt_template.invoke({"name":"유관순"})
res = model.invoke(prompt)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "사람의 이름", "title": "Name", "type": "string"}, "yob": {"description": "name의 사람이 사망한 년도", "title": "Yob", "type": "integer"}, "profile": {"description": "name의 사람에 대한 정보(profile)", "title": "Profile", "type": "string"}}, "required": ["name", "yob", "profile"]}
```


In [60]:
print(res.content)
result = parser.invoke(res)
print(type(result))
print(result)

```json
{
  "name": "유관순",
  "yob": 1920,
  "profile": "유관순은 대한민국의 독립운동가로, 1919년 3.1 운동에 참여하여 독립을 위해 싸운 인물입니다. 그녀는 특히 3.1 운동 당시의 만세 시위에서 주도적인 역할을 하였으며, 이후 일본 경찰에 체포되어 투옥되었습니다. 유관순은 독립운동의 상징적인 인물로, 그의 희생과 용기는 한국 역사에서 중요한 의미를 가지며, 현재도 많은 사람들에게 기억되고 있습니다."
}
```
<class '__main__.Person'>
name='유관순' yob=1920 profile='유관순은 대한민국의 독립운동가로, 1919년 3.1 운동에 참여하여 독립을 위해 싸운 인물입니다. 그녀는 특히 3.1 운동 당시의 만세 시위에서 주도적인 역할을 하였으며, 이후 일본 경찰에 체포되어 투옥되었습니다. 유관순은 독립운동의 상징적인 인물로, 그의 희생과 용기는 한국 역사에서 중요한 의미를 가지며, 현재도 많은 사람들에게 기억되고 있습니다.'


## YamlOutputParser
- YAML 형태로 받은 응답을 pydantic Model 객체로 반환한다.

In [65]:
from langchain.output_parsers import YamlOutputParser


# pydantic으로 JSON schema 를 정의
class Person(BaseModel):
    name: str = Field(description="사람의 이름")
    yod: int = Field(description="name의 사람이 태어난 년도")
    yod: int = Field(description="name의 사람이 사망한 년도")
    profile: str = Field(description="name의 사람에 대한 정보(profile)")

parser = YamlOutputParser(pydantic_object=Person)
print(parser.get_format_instructions())         
# JsonoutputParser와 동일한 guide. JSON 으로 응답 -> Pydantic Model 객체로 변환.

prompt_template = PromptTemplate(
    template="{name}에 대해서 설명해주세요. \n{format_instruction}",
    partial_variables={"format_instruction": parser.get_format_instructions()}
)

prompt = prompt_template.invoke({"name":"이순신"})
res = model.invoke(prompt)

The output should be formatted as a YAML instance that conforms to the given JSON schema below.

# Examples
## Schema
```
{"title": "Players", "description": "A list of players", "type": "array", "items": {"$ref": "#/definitions/Player"}, "definitions": {"Player": {"title": "Player", "type": "object", "properties": {"name": {"title": "Name", "description": "Player name", "type": "string"}, "avg": {"title": "Avg", "description": "Batting average", "type": "number"}}, "required": ["name", "avg"]}}}
```
## Well formatted instance
```
- name: John Doe
  avg: 0.3
- name: Jane Maxfield
  avg: 1.4
```

## Schema
```
{"properties": {"habit": { "description": "A common daily habit", "type": "string" }, "sustainable_alternative": { "description": "An environmentally friendly alternative to the habit", "type": "string"}}, "required": ["habit", "sustainable_alternative"]}
```
## Well formatted instance
```
habit: Using disposable water bottles for daily hydration.
sustainable_alternative: Switch t

In [66]:
print(res.content)

```
name: 이순신
yod: 1545
profile: 조선 중기의 무신이자 뛰어난 해군 전략가로, 임진왜란 동안 일본과의 전투에서 수많은 승리를 이끌어냈습니다.
```


In [67]:
result = parser.invoke(res)

print(type(result))
result
# pydantic Model 객체로 반환됨

<class '__main__.Person'>


Person(name='이순신', yod=1545, profile='조선 중기의 무신이자 뛰어난 해군 전략가로, 임진왜란 동안 일본과의 전투에서 수많은 승리를 이끌어냈습니다.')

In [68]:
result.name, result.yod 

('이순신', 1545)